<a href="https://colab.research.google.com/github/saurabhsingh1411/twitter_sentiment_analysis/blob/master/sentimantal%20analysis/twitter_sentimental_analysis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing lib and dataset

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [3]:
data=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


In [4]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [5]:
data = data[['tweet','label']]
data.head()

,tweet,label
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,0
1,Finally a transparant silicon case ^^ Thanks t...,0
2,We love this! Would you go? #talk #makememorie...,0
3,I'm wired I know I'm George I was made that wa...,0
4,What amazing service! Apple won't even talk to...,1


In [12]:
data = data[data.label != "Neutral"]
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['label'] == 1].size)
print(data[ data['label'] == 0].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

4052
11788


In [16]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,SpatialDropout1D,Dense

In [17]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 48, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 48, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
Y = pd.get_dummies(data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5306, 48) (5306, 2)
(2614, 48) (2614, 2)


In [23]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
166/166 - 46s - loss: 0.3476 - accuracy: 0.8464
Epoch 2/7
166/166 - 42s - loss: 0.2239 - accuracy: 0.9084
Epoch 3/7
166/166 - 42s - loss: 0.1851 - accuracy: 0.9254
Epoch 4/7
166/166 - 43s - loss: 0.1529 - accuracy: 0.9382
Epoch 5/7
166/166 - 43s - loss: 0.1306 - accuracy: 0.9499
Epoch 6/7
166/166 - 43s - loss: 0.1045 - accuracy: 0.9583
Epoch 7/7
166/166 - 43s - loss: 0.0881 - accuracy: 0.9676


In [24]:
twt=test['tweet']

In [27]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['tweet'].values)
X_test2 = tokenizer.texts_to_sequences(test['tweet'].values)
X_test2 = pad_sequences(X)

In [25]:
#twt = tokenizer.texts_to_sequences(twt)

In [26]:
#twt = pad_sequences(twt, maxlen=120, dtype='int32', value=0)

In [28]:
X_test2

array([[   0,    0,    0, ...,  136,  129,    1],
       [   0,    0,    0, ...,   17,  304,  279],
       [   0,    0,    0, ...,  146,  336, 1238],
       ...,
       [   0,    0,    0, ...,  168,   39,    4],
       [   0,    0,    0, ...,    7,  343,  465],
       [   0,    0,    0, ...,  105, 1913,   52]], dtype=int32)

In [29]:
#prediction=np.argmax(model.predict(X_test2))

In [33]:
prediction_final=np.argmax(model.predict(X_test2), axis=-1)

In [35]:
prediction_final[1:10]

array([0, 0, 0, 1, 1, 0, 0, 0, 0])

In [ ]:
print(train.shape)
print(test.shape)

(7920, 3)
(1953, 2)


In [ ]:
test['label']='test'
test.head()

,id,tweet,label
0,7921,I hate the new #iphone upgrade. Won't let me d...,test
1,7922,currently shitting my fucking pants. #apple #i...,test
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",test
3,7924,My ipod is officially dead. I lost all my pict...,test
4,7925,Been fighting iTunes all night! I only want th...,test


In [ ]:
data=pd.concat([train,test],ignore_index=True)

In [ ]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [ ]:
tweet=data['tweet']
sentiment=data['label']


## pereprocessing and fitting 

In [ ]:
print(type(tweet))
print(type(sentiment))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
vocab_size=10000
embedding_dim=16
max_length=120
truc_type='post'
oov_tok='<OOV>'

In [ ]:
def preprocessing(tweet):

  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences


  tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
  tokenizer.fit_on_texts(tweet)
  word_index=tokenizer.word_index
  sequences=tokenizer.texts_to_sequences(tweet)
  padded=pad_sequences(sequences,maxlen=max_length,truncating=truc_type)

  return padded

In [ ]:
tweet=train['tweet']

padded=preprocessing(tweet)

padded

array([[   0,    0,    0, ...,  159,  154,    2],
       [   0,    0,    0, ...,    4,    6, 7101],
       [   0,    0,    0, ...,  196,   18, 7103],
       ...,
       [   0,    0,    0, ...,   50,    9, 3093],
       [   0,    0,    0, ...,  175,    1,    1],
       [   0,    0,    0, ...,    4,    6,    1]], dtype=int32)

In [ ]:
# label=train['label']
# label

0       0
1       0
2       0
3       0
4       1
       ..
7915    0
7916    0
7917    0
7918    0
7919    0
Name: label, Length: 7920, dtype: int64

In [ ]:
# X=padded 
# y=np.array(label)

# from keras.utils import to_categorical

# y=to_categorical(y)

# print(type(X))
# print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
# print(X.shape)
# print(y.shape)

(7920, 120)
(7920, 2)


In [ ]:
X[0:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 2177, 7097, 1580,   20,  165,  157,
        7098,   43,   94,   77,   53,  172,  152,  159,  154,    2]],
      dtype=int32)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=21)

In [ ]:
# X_train = np.reshape(X_train, X_train.shape + (1,))
# X_test = np.reshape(X_test, X_test.shape + (1,))

In [ ]:
# print(X_train.shape, X_test.shape)
# print(y_train.shape, y_test.shape)

(7128, 120, 1) (792, 120, 1)
(7128, 2) (792, 2)


In [ ]:
#print(X_train.shape,type(X_train))
#print(X_test.shape,type(X_test))

In [ ]:
# import tensorflow as tf 

# model=tf.keras.Sequential([
#                            tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
#                            #tf.keras.layers.GlobalAveragePooling2D(),
#                            tf.keras.layers.Dropout(rate=0.2),
#                            tf.keras.layers.Dense(6,activation='relu'),
#                            tf.keras.layers.Dropout(rate=0.2),
#                            tf.keras.layers.Dense(6,activation='relu'),
#                            tf.keras.layers.Dropout(rate=0.2),
#                            tf.keras.layers.Dense(2,activation='sigmoid')
# ])

# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# model.summary()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
embed_dim = 128
lstm_out = 196
max_fatures=2000

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 120, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(train['label']).values
X=padded

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6336, 120) (6336, 2)
(1584, 120) (1584, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7


InvalidArgumentError: ignored

In [ ]:
# model = Sequential()
# model.add(Dropout((0.25), batch_input_shape=(None,2092,1)))
# model.add(Conv1D(128, 8, use_bias=True, activation='relu', bias_initializer='zeros', batch_input_shape=(None,2092,1)))
# #model.add(LeakyReLU(alpha=0.3))
# #model.add(Dropout(0.2))
# model.add(Conv1D(64, 4, use_bias=True, activation='relu', bias_initializer='zeros', batch_input_shape=(None,2092,1)))
# #model.add(LeakyReLU(alpha=0.3))
# model.add(BatchNormalization(momentum=0.99, epsilon=0.0005))
# model.add(Flatten())
# model.add(Dropout(0.25))
# model.add(Dense(128, activation='relu'))                # may use input_dim=(X,) if want to
# #model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(2, activation='sigmoid'))
# sgd = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# Use callbacks to optimize and control the learning process
#es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=10)
#ch = ModelCheckpoint('comb_bestmodel.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=2)
#relr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, min_lr=0.0001, mode='min', verbose=1, min_delta=1E-6)
#callbacks_list = [es,ch,relr]

In [ ]:
# Train model on the traning dataset
history = model.fit(X_train,y_train,epochs=100, verbose=2, shuffle=True, batch_size=64, validation_split=0.1) #callbacks=callbacks_list)

Epoch 1/100


ValueError: ignored

In [ ]:
num_epochs=100
model.fit(X_train,y_train,epochs=num_epochs,validation_data=(X_test,y_test))

Epoch 1/100


ValueError: ignored

prediction on text

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
test.head()

In [ ]:
tweet_test=test['tweet']
tweet_test.head()

In [ ]:
tweet_test=preprocessing(tweet_test)

In [ ]:
y_pred=model.predict(tweet_test)

In [ ]:
y_pred[0:1]

In [ ]:
prediction=np.argmax(model.predict(tweet_test), axis=-1)

In [ ]:
print(prediction.shape)
print(test.shape)

In [ ]:
prediction[0:6]

In [ ]:
#predict_label=model.predict(tweet_test)

In [ ]:
#predict_label=np.argmax(predict_label,axis=-1)

In [ ]:
#predict_label=model.predict_classes(tweet_test)

#predict_label